In [1]:
import os
import pandas as pd
from pathlib import Path

# 1️⃣ Ruta a la carpeta "data" (sube un nivel desde notebooks y entra en data)
base_path = Path.cwd().parent / "data/processed"

# 2️⃣ Buscar todos los archivos CSV dentro de la carpeta (también comprimidos .gz)
csv_files = []
for pattern in ("*.csv", "*.csv.gz"):
    csv_files.extend(base_path.glob(pattern))

if not csv_files:
    raise FileNotFoundError(f"No se encontraron archivos CSV en {base_path}")

# 3️⃣ Seleccionar el CSV más reciente por fecha de modificación
latest_csv = max(csv_files, key=os.path.getmtime)
print(f"📂 Cargando archivo más reciente: {latest_csv.name}")

# 4️⃣ Cargarlo en Pandas (compression se infiere por extensión)
df = pd.read_csv(latest_csv, low_memory=False)

# 5️⃣ Vista rápida de datos
print(f"✅ Archivo cargado con {df.shape[0]:,} filas y {df.shape[1]} columnas.")
print(df.head())


📂 Cargando archivo más reciente: consulta_01_2025-09-12_12-13_v01.csv
✅ Archivo cargado con 9,484 filas y 61 columnas.
                              device_id  SerialNumber  ...         lon        lat
0  8c0ca7b2-0400-4d7b-8294-a0b3e461ada7      70010002  ...    2.557738  41.605218
1  dce17e04-1913-41b1-96e2-a5deb9fa0e75      80000237  ... -101.304733  26.374407
2  62c88758-64fd-4ea1-9054-2da2a17a8353      80000312  ... -101.304558  26.374458
3  f39d5016-f727-47aa-a50c-27a1d4cc47cd      80000527  ... -101.287781  26.427490
4  d0d6d480-cbe0-4f22-8198-f842519fe9c9      80000280  ... -101.308846  26.380205

[5 rows x 61 columns]


In [6]:
print(cols_info.to_string())


                                                     indice                                              columna tipo_dato
device_id                                                 0                                            device_id    object
SerialNumber                                              1                                         SerialNumber     int64
Model                                                     2                                                Model    object
mensajes_esperados                                        3                                   mensajes_esperados     int64
mensajes_recibidos                                        4                                   mensajes_recibidos     int64
mensajes_sin_gps                                          5                                     mensajes_sin_gps     int64
pct_recibidos_vs_esperados                                6                           pct_recibidos_vs_esperados   float64
pct_sin_gps_vs_e

In [3]:
# Filtrar por ranch_name = "Daniel Arias"
df_consulta_ganaderia = df[df["ranch_name"] == "Daniel Arias González"].copy()

# Ver primeras filas para comprobar
print(df_consulta_ganaderia.head())


                                 device_id  SerialNumber  ...       lon        lat
852   0de8dd0b-c699-464d-9558-c2265367cf21      70014324  ... -6.403479  43.181114
4485  244a7b73-e270-4a76-96ce-6c307daac840      70014332  ... -6.431961  43.203197
4913  efff585d-444a-43b8-9e2b-31a27d27016c      70014321  ... -6.431895  43.203220
5228  541d140b-46fc-4be9-8b28-cc1807800bd1      70014327  ... -6.431937  43.203281
5303  6b3e8a41-f3bf-40ed-b7c2-95be0d525911      70013775  ... -6.431746  43.203060

[5 rows x 61 columns]


In [4]:
import pandas as pd
import numpy as np

# 1) Filtro del cliente
df_consulta_ganaderia = df[df["ranch_name"].str.strip().str.upper() == "DANIEL ARIAS"].copy()

# Si tienes tabla de alertas:
if "df_alertas" in globals():
    al = df_alertas.copy()
    al = al[al["alert_type"].str.lower().eq("baja_actividad")]
    al = al.merge(df_consulta_ganaderia[["device_id","animal_name"]].drop_duplicates(),
                  on=["device_id","animal_name"], how="inner")

    al["fecha"] = pd.to_datetime(al["created_at"]).dt.date

    # Volumen por día y animal
    al_dia = (al.groupby(["fecha","animal_name"], as_index=False)
                .size()
                .rename(columns={"size":"n_alertas_dia"}))

    # KPI resumen últimos 30 días
    ultimos_30 = pd.Timestamp.today().normalize() - pd.Timedelta(days=30)
    al_30 = al[pd.to_datetime(al["created_at"]) >= ultimos_30]
    kpi = (al_30.groupby("animal_name", as_index=False)
                .size()
                .rename(columns={"size":"n_alertas_30d"})
                .sort_values("n_alertas_30d", ascending=False))

    # ¿≥8 alertas/día?
    picos = al_dia[al_dia["n_alertas_dia"] >= 8].sort_values(["fecha","n_alertas_dia"], ascending=False)

else:
    # 2) Aproximación si NO tienes tabla de alertas:
    # Necesitas una métrica de "actividad" o "desplazamiento" por ventana (ej. distancia, varianza acelerómetro, etc.)
    # Aquí invento un proxy simple: 'dist_m' ya calculada por uplink, si la tienes:
    df_g = df_consulta_ganaderia.copy()
    df_g["timestamp"] = pd.to_datetime(df_g["timestamp"])

    # Ventana de 1h: actividad media
    win = "1H"
    act = (df_g
           .set_index("timestamp")
           .groupby("animal_name")["dist_m"]
           .rolling(win).mean()
           .rename("act_1h")
           .reset_index())

    df_g = df_g.merge(act, on=["animal_name","timestamp"], how="left")

    # Baseline "global" (histórico largo) por animal
    base = (df_g.groupby("animal_name", as_index=False)["act_1h"]
                .median().rename(columns={"act_1h":"baseline_hist_mediana"}))

    df_g = df_g.merge(base, on="animal_name", how="left")

    # Regla simple de baja actividad (proxy):
    # alerta si act_1h < 30% del baseline histórico
    df_g["alerta_proxy"] = df_g["act_1h"] < (0.30 * df_g["baseline_hist_mediana"])

    # Conteo diario de alertas-proxy
    df_g["fecha"] = df_g["timestamp"].dt.date
    al_dia = (df_g.groupby(["fecha","animal_name"], as_index=False)
                .agg(n_alertas_dia=("alerta_proxy","sum")))

    picos = al_dia[al_dia["n_alertas_dia"] >= 8].sort_values(["fecha","n_alertas_dia"], ascending=False)
    kpi = (al_dia.groupby("animal_name", as_index=False)
                .agg(n_alertas_30d=("n_alertas_dia","sum"))
                .sort_values("n_alertas_30d", ascending=False))

# === Calidad de datos (posibles causas de falsos positivos) ===
# % sin GPS, reinicios, batería baja por animal en últimos 30 días
ref_30 = pd.Timestamp.today().normalize() - pd.Timedelta(days=30)
q = df_consulta_ganaderia[pd.to_datetime(df_consulta_ganaderia["ultimo_mensaje_recibido"]) >= ref_30].copy()

def pct(x):
    return 100 * x.mean()

calidad = (q.groupby("animal_name", as_index=False)
             .agg(
                n_msgs=("mensajes_recibidos","sum"),
                pct_sin_gps=("mensajes_sin_gps", lambda s: 100 * s.sum() / max(1, q.loc[s.index, "mensajes_recibidos"].sum())),
                reinicios=("numero_reinicios","sum"),
                batt_med=("porcentaje_bateria","mean")
             ))

# === Simulación: "reseteo" del baseline con ventana reciente ===
# Ejemplo: baseline reciente = mediana de actividad de los últimos 7 días por animal
if "act" in locals():  # existe df_g con 'act_1h'
    recent_from = pd.Timestamp.today().normalize() - pd.Timedelta(days=7)
    recent = df_g[df_g["timestamp"] >= recent_from].copy()
    base_recent = (recent.groupby("animal_name", as_index=False)["act_1h"]
                        .median()
                        .rename(columns={"act_1h":"baseline_reciente_mediana"}))

    df_sim = df_g.merge(base_recent, on="animal_name", how="left")

    # Nueva regla con baseline reciente (ej.: 40% del baseline reciente)
    df_sim["alerta_proxy_reset"] = df_sim["act_1h"] < (0.40 * df_sim["baseline_reciente_mediana"])

    sim_dia = (df_sim.groupby([df_sim["timestamp"].dt.date, "animal_name"], as_index=False)
                    .agg(n_alertas_reset=("alerta_proxy_reset","sum"))
                    .rename(columns={"timestamp":"fecha"}))

    # Comparativa por día/animal (antes vs después)
    comp = (al_dia.merge(sim_dia, on=["fecha","animal_name"], how="left")
                 .fillna({"n_alertas_reset":0})
                 .assign(delta=lambda d: d["n_alertas_reset"] - d["n_alertas_dia"])
                 .sort_values(["fecha","delta"]))


KeyError: 'timestamp'